<h1 style="text-align:center">
    <font "color = #2D3A39";"size=20">
        <span style="font-family:Helvetica Neue Condensed">
            Youtube US Trending Videos Explorative Data Analysis
        </span> 
    </font>
</h1>
<hr style="width:100%;height:5px;border-width:0;color:gray;background-color:gray">
<center><img src="https://boostlikes-bc85.kxcdn.com/blog/wp-content/uploads/2017/09/Trending-on-YouTube.jpg"></center>

<h2>   
      <font color = blue ; size= 6>
            <span style='font-family:Georgia'>
            Table of Contents:
            </span>   
        </font>    
</h2>
<span style='font-family:Georgia'>
    <ul>
        <li><a href='#intro'>1. Introduction</a></li>
    </ul>
    <ul>
        <li><a href='#reading'>2. Reading and Manipulating the Data</a></li>
        <ul>
        <li><a href='#import'>2.1 Libraries and Functions</a></li>
        <li><a href='#data'>2.2 Import Data</a></li> 
        <li><a href='#properties'>2.3 Data Properties</a></li>
            <ul>
            <li><a href='#nulls'>2.3.1 Null Evaluation and Imputation</a></li>
            </ul>
           </ul>
        </ul>
        <ul>
        <li><a href='#eda'>3. Exploratory Data Analysis</a></li>
        <ul>
        <li><a href='#topcat'>3.1 Overall Top Trending Videos by Likes, Views and Comments</a></li>
        <li><a href='#corr'>3.2 Correlation Between Trending Video Properties</a></li>
        <li><a href='#wordcloud'>3.3 Wordcloud Analysis</a></li>
        <li><a href='#disliked'>3.4 Most Disliked or Controversial Trending Videos</a></li>
        <li><a href='#repeat'>3.5 Repeat Trending Videos and Channels</a></li>
        <li><a href='#delta'>3.6 Time Passed Since Published</a></li>
        </ul> 
     </ul>      
</span>

<a id="intro"></a>
<h2>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            1. Introduction
            </span>   
        </font>    
</h2>

Trending videos on youtube are popular videos which appear on the main page.<br> Trending videos are selected, according to Google by answering several criterias:

* Are appealing to a wide range of viewers
* Are not misleading, clickbaity or sensational
* Capture the breadth of what’s happening on YouTube and in the world
* Showcase a diversity of creators
* Ideally, are surprising or novel

In addition they are determined by:

* View count
* How quickly the video is generating views (i.e. “temperature”)
* Where views are coming from, including outside of YouTube
* The age of the video
* How the video performs compared to other recent uploads from the same channel

In this analysis we will be diving into the different properties of trending videos.<br> We will explore the relationship between the different parameters and try to understand what type of genres have a place in the youtube Trending section in the US.<br> 
This notebook is compatible with mounting the data from the other countries as well.

<a id="reading"></a>
<h2>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            2. Reading and Manipulating the Data
            </span>   
        </font>    
</h2>

<a id="import"></a>
<h3>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            2.1 Import python Libraries and define functions
            </span>   
        </font>    
</h3>

In [ ]:
#import pandas relevant libraries
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#display numbers without scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x) 

In [ ]:
#Define functions for this dataset
#define function to replace null values. compatible with the rest of the yt datasets
def null_desc(df):
    """
    replace null video descriptions
    with 'No Description'
    """
    
    df['description'].replace(np.nan, 'No Description', inplace=True)
    
    print ('Null count\n{}'.format(df.isnull().sum()))

#define function to drop duplicate titles, keeping first or most recent trending date
def title_dup_rmv(df,kp):
    """
    drop duplicate titles
    and keep the data from
    the last or first posted date
    use 'first' or 'last' as kp
    """
    clean_df= df.sort_values('trending_date', ascending=True).drop_duplicates('title', keep=kp)
    clean_df= clean_df.sort_values('trending_date', ascending=False)
    return(clean_df)

<a id="data"></a>
<h3>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            2.2 Import Data
            </span>   
        </font>    
</h3>

In [ ]:
filep = '../input/youtube-trending-video-dataset/US_youtube_trending_data.csv'
yt_us = pd.read_csv(filep,
                   parse_dates=['publishedAt','trending_date'])
yt_us.head()

<a id="properties"></a>
<h3>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            2.3 Data Properties
            </span>   
        </font>    
</h3>

In [ ]:
yt_us.info()

In [ ]:
yt_us.describe()

<a id="nulls"></a>
<h3>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            2.3.1 Null Evaluation and Imputation
            </span>   
        </font>    
</h3>

In [ ]:
#display how many nulls are in each column
px.bar(yt_us.isnull().sum()
       ,title='Nulls Per Column'
       ,color=yt_us.isnull().sum().index)

<div class="alert alert-block alert-info">
    <font color = #2D3A39 ; size=4>
    <span style='font-family:Georgia, Times, Times New Roman'>
        <b>Results: </b><br>
        We see that the only columns with nulls is the description columns, indicating that some
        videos simply have no description input by the user. In this case we will simply replace
        the nulls with "No Description"
    </span>
    </font>
</div>

In [ ]:
#replace all null description with "No Description"
null_desc(yt_us)

<a id="eda"></a>
<h2>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            3. Exploratory Data Analysis
            </span>   
        </font>    
</h2>

<a id="topcat"></a>
<h3>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            3.1 Overall Top Trending Videos by Likes, Views and Comments
            </span>   
        </font>    
</h3>

In [ ]:
yt_top = title_dup_rmv(yt_us,'last') #remove duplicate titles, keeping the most recent trending date entry.
yt_top_l = yt_top.sort_values('likes', ascending= False)
yt_top_v = yt_top.sort_values('view_count',ascending = False)
yt_top_c = yt_top.sort_values('comment_count', ascending = False)

In [ ]:
#initialize interactive graph object
fig = go.Figure()

#add each of the graph traces for likes, views and comments
fig.add_trace(
    go.Bar(
        y=yt_top_l['title'].head(5),
        x=yt_top_l['likes'].head(5),
        hovertext=yt_top_l['channelTitle'],
        orientation='h',
        name= 'Likes',
        marker=dict(color = '#298523')))

fig.add_trace(
    go.Bar(
        y=yt_top_v['title'].head(5),
        x=yt_top_v['view_count'].head(5),
        hovertext=yt_top_v['channelTitle'],
        orientation='h',
        name= 'Views',
        visible=False,
        marker=dict(color = '#E38D1E')))

fig.add_trace(
    go.Bar(
        y=yt_top_c['title'].head(5),
        x=yt_top_c['comment_count'].head(5),
        hovertext=yt_top_c['channelTitle'],
        orientation='h',
        name= 'Comments',
        visible= False,
        marker= dict(color = '#1E39E3')))

#update layout with buttons, corresponding to each trace
fig.update_layout(
    {
    'title' : {
        'font_size' : 20,
        'text' : 'Top 5 Trending Videos by Likes, Views or Comments'}},
    yaxis = {'title' : 'Video Title'},
    updatemenus= [dict(
        type='buttons',
        direction='right',
        active=0,
        y=1.1,
        x=0.8,
        buttons=list([
            dict(label='Likes'
                ,method='update'
                ,args=[{'visible' : [True,False,False]},
                     {'title' : 'Top 5 Trending Videos by Likes',
                     'xaxis': {'title': 'Likes'}}]),
            dict(label='Views'
                ,method='update'
                ,args=[{'visible' : [False,True,False]},
                    {'title' : 'Top 5 Trending Videos by Views',
                     'xaxis': {'title': 'Views'}}]),
            dict(label='Comments'
                ,method='update'
                ,args=[{'visible' : [False,False,True]},
                {'title' : 'Top 5 Trending Videos by Comments',
                'xaxis': {'title': 'Comments'}}])
        ])
        )])
#display figure
fig.show()

<a id="corr"></a>
<h3>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            3.2 Correlation Between Trending Video Properties
            </span>   
        </font>    
</h3>

 We'll use Pearson correlation to examine the relation between likes, dislikes, views, and comments. <br>
 Since we know that some of the videos have comments and/or likes disabled we will remove them from this part of the analysis

In [ ]:
corr_us = yt_us[['title','channelTitle','view_count','likes',\
                 'dislikes','comment_count','comments_disabled',\
                 'trending_date','tags']].copy()

In [ ]:
#define comment filter for titles that have comments enabled but have 0 comments
cmt_err_mask = (corr_us['comments_disabled'] == False)&(corr_us['comment_count'] == 0)

#if erroneous entries are detected, here we drop all rows where comments are enabled AND the comment count is 0 
corr_us.drop(corr_us[cmt_err_mask].index, inplace=True)

#define filter for titles where comments are disabled
cmt_dis_mask = corr_us['comments_disabled'] == True
#drop rows where comments are disabled
corr_us.drop(corr_us[cmt_dis_mask].index, inplace=True)

#drop all rows with 0 likes
corr_us.drop(corr_us[corr_us['likes'] == 0].index, inplace=True)

#remove duplicate titles while keeping the most recent data about each video
corr_us = title_dup_rmv(corr_us,'last')


In [ ]:
#use pearson correlation
go_corr = corr_us[['view_count','likes','dislikes','comment_count']].corr()
go_corr

In [ ]:
#initialize plotly express figure
fig= px.imshow(go_corr
               ,title='Correlation Between Trending Youtube Videos Attributes')
fig.update_layout(
    title_font_family="Times New Roman",
    title_font_color="#2D3A39",
    font_size = 16)
fig.show()

<div class="alert alert-block alert-info">
    <font color = #2D3A39 ; size=4>
    <span style='font-family:Georgia, Times, Times New Roman'>
        <b>Insight: </b><br>
        we can see that there is a strong correlation between all the different values (0.5+), and specifically between
likes and view_count which is expected - typically users must view a video before liking/ disliking or commenting on it.<br>
we also see a strong correlation between comment_count and likes which indicates that people who like a video are more likely
to comment on it.
    </span>
    </font>
</div>

In [ ]:
#plotly express figure
px.scatter(corr_us,'likes','comment_count'
           ,title='Correlation between Comment Count and Likes in Trending Videos')

<a id="wordcloud"></a>
<h3>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            3.3 Wordcloud Analysis
            </span>   
        </font>    
</h3>

 We want to see which tags appear most in youtube trending videos

In [ ]:
#lets analyze the tags of trending videos
tags = yt_us.copy()
tags = title_dup_rmv(tags,'last') #remove duplicate titles
tags = tags[['tags']]

#we'll ignore videos without tags and join tags with space using '-'
mask = tags['tags'] != '[None]'
tags = tags['tags'][mask].str.replace(' ','-').to_frame()
tags = tags['tags'][mask].str.replace('|',' ')

#prepare the tags for word processing
text = list(set(tags)) 
wordcloud = WordCloud(max_font_size=50,
                      background_color='black',
                      scale=3,
                      max_words=20
                     ).generate(str(text))

In [ ]:
#initialize image for wordcloud word processing
fig = px.imshow(wordcloud)
fig.update_layout({
    'xaxis': {
        'visible': False},
    'yaxis': {
        'visible': False},
    'title': {
        'text' : 'Top 20 Tags from Trending Youtube Videos',
        'font_size' : 25
    }})
fig.show()


<div class="alert alert-block alert-info">
    <font color = #2D3A39 ; size=4>
    <span style='font-family:Georgia, Times, Times New Roman'>
        <b>Insight: </b><br>
           At the time of this analysis: we can see that the top tags are from:<br> gaming (Among Us) and music related tags such as Music Video,
            and Hip hop.
    </span>
    </font>
</div>


<a id="disliked"></a>
<h3>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            3.4 Most Disliked or Controversial Trending Videos
            </span>   
        </font>    
</h3>

 We want to see which videos were overwhelmingly disliked over liked, which we will call "Disliked".<br> 
 We also want to look at videos that had a balanced like to dislike ratio which we will call "Controversial".

In [ ]:
#create a copy of the dataframe for further manipulation
yt_disliked = yt_us[['likes','dislikes','title','trending_date','channelTitle']].copy()

In [ ]:
#define filters 
mask = yt_disliked['likes'] < yt_disliked['dislikes']

yt_disliked = yt_disliked[mask]
yt_disliked = title_dup_rmv(yt_disliked,'last') #remove duplicate titles
yt_disliked['ratio'] = yt_disliked['dislikes']/yt_disliked['likes'] #assign a 'ratio' column that shows the level of dislike

#define ratio filters
most_disliked = yt_disliked['ratio'] > 1.6
most_controversial = yt_disliked['ratio'] <=1.6

#shows the most disliked videos
dislike_data = yt_disliked[most_disliked]

#shows the most controversial videos
con_data = yt_disliked[most_controversial]

#define dislike threshold - only show titles that have over 200 dislikes
dd_dt = dislike_data['dislikes'] > 200 
cd_dt = con_data['dislikes'] > 200

#top 5 disliked and controversial videos
dislike_data = dislike_data[dd_dt].sort_values('ratio', ascending=False).head(5)
con_data = con_data[cd_dt].sort_values('ratio', ascending=False).head(5)

In [ ]:
#initialize interactive graph object
fig = go.Figure()

#add bar graph for most disliked videos
fig.add_trace(go.Bar(
    x= dislike_data['ratio']
    ,y= dislike_data['title']
    ,name='Top 5 Most Disliked Videos'
    ,orientation='h'
    ,hovertext= 'Channel Name: '+con_data['channelTitle']
    ,marker=dict(
        color=dislike_data['dislikes'],
        colorscale="sunsetdark",
        colorbar= {'title' : 'Dislikes'})))

#add bar graph for most controversial videos
fig.add_trace(go.Bar(
    x= con_data['ratio']
    ,y= con_data['title']
    ,name='Top 5 Most Controversial Videos'
    ,visible= False
    ,orientation='h'
    ,hovertext= 'Channel Name: '+con_data['channelTitle']
    ,marker=dict(
        color=dislike_data['dislikes'],
        colorscale="sunsetdark",
        colorbar= {'title' : 'Dislikes'})))

#update title and axis labels and assign buttons for the different traces
fig.update_layout({
    'title' : {'text' : 'Top 5 Most Disliked or Controversial Videos'},
    'xaxis' : {
        'title' : {'text':'Dislike/Like Ratio',
                  'font_size' : 17}},
    'yaxis' : {
        'title' : {'text' : 'Video Title',
                  'font_size' : 17}
    }},
    updatemenus= [dict(
    type='buttons',
    direction='right',
    active=0,
    x=0.9,
    y=1.12,
    buttons=list([
        dict(label='Most Disliked'
            ,method='update'
            ,args=[{'visible' : [True,False]},
                 {'title' : 'Top 5 Most Disliked Videos'}]),
        dict(label='Most Controversial'
            ,method='update'
            ,args=[{'visible' : [False,True]},
                {'title' : 'Top 5 Most Controversial Videos'}])
    ])
)])

fig.show()

<div class="alert alert-block alert-info">
    <font color = #2D3A39 ; size=4>
    <span style='font-family:Georgia, Times, Times New Roman'>
        <b>Results:</b><br>
        As we can see, the reasons these videos are not recieved well is sometimes obvious from the title and sometimes rather
        obscure.<br> I encourage you to check some of these out, it's rather fascinating finding out why they are so hated or controversial! 
    </span>
    </font>
</div>


<a id="repeat"></a>
<h3>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            3.5 Repeat Trending Videos and Channels
            </span>   
        </font>    
</h3>

 While working with the data we've seen that many videos trend multiple times.<br>
 We want to check which videos and which channels trended the most times.

In [ ]:
#top 5 repeat trended videos
trend_vd = yt_us[['title','channelTitle']].\
    assign(trended_count= 1).\
    groupby(['title','channelTitle'],as_index=False).\
    count().\
    sort_values('trended_count',ascending=False).head(5).sort_values('trended_count')

In [ ]:
#top 5 repeat trended channels
trend_ch = yt_us[['channelTitle']].\
    assign(trended_count= 1).\
    groupby(['channelTitle'], as_index=False).\
    count().\
    sort_values('trended_count',ascending=False).head(5).sort_values('trended_count')

In [ ]:
#initialize interactive graph object
fig = go.Figure()

#add bar graph for top 5 trended videos
fig.add_trace(go.Bar(
    x= trend_vd['trended_count']
    ,y= trend_vd['title']
    ,visible= False
    ,orientation='h'
    ,hovertext= 'Channel Name: '+trend_vd['channelTitle']
    ,marker=dict(color = '#CF5818')))

#add bar graph for top 10 repeat trended channels
fig.add_trace(go.Bar(
    x= trend_ch['trended_count']
    ,y= trend_ch['channelTitle']
    ,orientation='h'
    ,marker=dict(color = '#228EB7')))

#update graph layout and add buttons for each trace
fig.update_layout({
    'title' : {
        'font_size' : 20,
        'text' : 'Top 5 Repeated Trending Channels and Videos'},
    'xaxis' : {
        'title' : {'text':'Number of Times Trended',
                  'font_size' : 17}},
    'yaxis' : {
        'title' : {'text' : 'Video Title',
                  'font_size' : 17}
    }},
    updatemenus= [dict(
    type='buttons',
    direction='right',
    active=1,
    x=1,
    y=1.12,
    buttons=list([
        dict(label='Videos'
            ,method='update'
            ,args=[{'visible' : [True,False]},
                 {'title' : 'Top 5 Repeating Trended Videos'}]),
        dict(label='Channels'
            ,method='update'
            ,args=[{'visible' : [False,True]},
                {'title' : 'Top 5 Repeating Trended Channels'}])
    ])
    )])

fig.show()

<div class="alert alert-block alert-info">
    <font color = #2D3A39 ; size=4>
    <span style='font-family:Georgia, Times, Times New Roman'>
        <b>At the time of this analysis: </b><br>
        We can see, the most repeat trended videos is the Starlink Mission video which is related to major relevant news, we also see music videos, sport events and social media personalities.<br>
        In the channel category we see multiple sport related channels, a gaming channel and its counterpart vlog channel.
    </span>
    </font>
</div>

<a id="delta"></a>
<h3>   
      <font color = #2D3A39 >
            <span style='font-family:Georgia'>
            3.6 Time Passed Since Published
            </span>   
        </font>    
</h3>

 We know, by google's criteria, that the age of a video affects the likelihood of it becoming a trending video.<br>
 In this part of the analysis we will check how many days passed since publishing of the video up to the date
 it trended for the first time, as well as its most recent trending date.

In [ ]:
### Last trending date will be marked with dtl

#remove duplicate titles and keep the most recent data from each title
dtl_c = title_dup_rmv(yt_us,'last')

#remove d,h,m from published date to match trending date data format.
dtl_comp = dtl_c[['trending_date','publishedAt']].\
                assign(date_gap = dtl_c['trending_date'] - pd.to_datetime(dtl_c['publishedAt']).\
                dt.floor('d')).\
                sort_values('date_gap', ascending=False)

#convert days calculation to int
dtl_comp['date_gap'] = dtl_comp.date_gap.apply(lambda x: x.days)

#calculate mean value and standard deviation
dtl_md = dtl_comp['date_gap'].mean()
dtl_std = dtl_comp['date_gap'].std()

#group by the date gap (days passed from publication up to the MOST RECENT trending date)
dtl_dist = dtl_comp[['date_gap','trending_date']]\
                    .groupby('date_gap', as_index=False)\
                    .count()\
                    .rename(columns= {'trending_date' : 'count'})


### First trending date will be marked with dtf

#remove duplicate titles and keep the first occurring entry from each title
dtf_c = title_dup_rmv(yt_us,'first')

#remove d,h,m from published date to match trending date data format.
dtf_comp = dtf_c[['trending_date','publishedAt']]\
                .assign(date_gap = dtf_c['trending_date'] - pd.to_datetime(dtf_c['publishedAt'])\
                .dt.floor('d'))\
                .sort_values('date_gap', ascending=False)

#convert days calculation to int
dtf_comp['date_gap'] = dtf_comp.date_gap.apply(lambda x: x.days)

#calculate mean value and standard deviation
dtf_md = dtf_comp['date_gap'].mean()
dtf_std = dtf_comp['date_gap'].std()

#group by the date gap (days passed from publication up to the FIRST trending date)
dtf_dist = dtf_comp[['date_gap','trending_date']]\
                    .groupby('date_gap', as_index=False)\
                    .count()\
                    .rename(columns= {'trending_date' : 'count'})


In [ ]:
#initialize interactive graph object
fig = go.Figure()

#add bar graph for last trending date
fig.add_trace(go.Bar(
    x=dtl_dist['date_gap'],
    y=dtl_dist['count'],
    visible= False,
    marker=dict(
        color=dtl_dist['date_gap'],
        cmin=dtl_md-dtl_std,
        cmax=dtl_md+dtl_std,
        cmid=dtl_md,
        colorbar= {'title' : 'Distance From Mean'},
        colorscale=[
            [0,'rgb(195, 155, 206)'],
            [0.25,'rgb(253, 109, 114)'],
            [0.5,'rgb(235, 116, 30)'],
            [0.75,'rgb(253, 109, 114)'],
            [1,'rgb(195, 155, 206)']])
))

#add bar graph for first trending date
fig.add_trace(go.Bar(
    x=dtf_dist['date_gap'],
    y=dtf_dist['count'],
    marker=dict(
        color=dtf_dist['date_gap'],
        cmin=dtf_md-dtf_std,
        cmax=dtf_md+dtf_std,
        cmid=dtf_md,
        colorbar= {'title' : 'Distance From Mean'},
        colorscale=[
            [0,'rgb(195, 155, 206)'],
            [0.25,'rgb(253, 109, 114)'],
            [0.5,'rgb(235, 116, 30)'],
            [0.75,'rgb(253, 109, 114)'],
            [1,'rgb(195, 155, 206)']])
))

#add indicator for last trending date mean 
fig.add_trace(go.Indicator(
    mode = "number",
    value = dtl_md,
    visible= False,
    title = {'text': "Mean",'font': {'color': 'red','size':20}},
    number={'font':{'color': 'gray','size':30}},
    domain = {'x': [0.8,0.9], 'y': [0.5,1]}
))

#add indicator for last trending date std 
fig.add_trace(go.Indicator(
    mode = "number",
    value = dtl_std,
    visible= False,
    title = {'text': "Standard Deviation",'font': {'color': 'red','size':20}},
    number={'font':{'color': 'gray','size':30}},
    domain = {'x': [0.8,0.9], 'y': [0.2,0.8]}
))

#add indicator for first trending date mean 
fig.add_trace(go.Indicator(
    mode = "number",
    value = dtf_md,
    title = {'text': "Mean",'font': {'color': 'red','size':20}},
    number={'font':{'color': 'gray','size':30}},
    domain = {'x': [0.8,0.9], 'y': [0.5,1]}
))

#add indicator for first trending date std
fig.add_trace(go.Indicator(
    mode = "number",
    value = dtf_std,
    title = {'text': "Standard Deviation",'font': {'color': 'red','size':20}},
    number={'font':{'color': 'gray','size':30}},
    domain = {'x': [0.8,0.9], 'y': [0.2,0.8]}
))

#update graph layout and add buttons for each trace
fig.update_layout(
    title='Days Passed From Publication to Date Trending - First and Last',
    xaxis_title='Days Elapsed Since Publication',
    yaxis_title='Total Videos',
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1,
        tickangle = 0),
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 500),
    updatemenus= [dict(
    type='buttons',
    direction='right',
    active=1,
    x=1,
    y=1.12,
    buttons=list([
        dict(label='Last Trending Date'
            ,method='update'
            ,args=[{'visible' : [True,False,True,True,False,False]},
                 {'title' : 'Days Passed From Publication to Last Day Trended'}]),
        dict(label='First Trending Date'
            ,method='update'
            ,args=[{'visible' : [False,True,False,False,True,True]},
                {'title' : 'Days Passed From Publication to First Day Trended'}])]))])

fig.show()

<div class="alert alert-block alert-info">
    <font color = #2D3A39 ; size=4>
    <span style='font-family:Georgia, Times, Times New Roman'>
        <b>Insight: </b><br>
        We can see that the first trending date for most videos is usually within 1 day of publication.<br>
        We also see that the last trending date seems to be between 5 and 7 days, which shows us that most videos that trend
        keep trending for around a week before they are no longer eligible for the trending section.
    </span>
    </font>
</div>

<hr style="width:100%;height:5px;border-width:0;color:gray;background-color:gray">

<h1 style="text-align:center">
    <font "color = #2D3A39";"size=20">
        <span style="font-family:Helvetica Neue Condensed">
            Thank You!
        </span> 
    </font>
</h1>


<div>
    <font color = #2D3A39 ; size=4>
        <span style='font-family:Georgia, Times, Times New Roman'>
            Thank you for taking the time to peruse this analysis!<br>
            It was very interesting to see the different facets of the data and I hope you found it interesting as well!<br>
            Any feedback is welcome.
        </span> 
    </font>
</div>